# NDA Data Upload

## Overview

Upload Steps 
+ All data in BIDS format
+ Subject info entered into excel sheet (need DOB, session dates, gender, task, etc)
+ Enter our subject id (ie 10001) and GUID from NDA tool into mapping.txt file
+ Run nda_upload.py script on thalamege to prepare data for upload
+ Upload fMRI, Diffusion, Resting-state data via HTML validation tool on NDA site (https://nda.nih.gov/vt/) or if EEG data, you must use nda_upload.py script to upload

**I have put most of the nda uploads at /mnt/nfs/lss/lss_kahwang_hpc/data/NDA_uploads**

## Upload Script

### Installing packages

pip3 install -r /mnt/nfs/lss/lss_kahwang_hpc/data/NDA_uploads/requirements.txt

The packages in that file are listed below

cffi==1.15.0   
cryptography==36.0.1  
et-xmlfile==1.1.0  
future==0.18.2  
glob2==0.7  
msoffcrypto-tool==5.0.0  
natsort==8.1.0  
nibabel==3.2.2  
numpy  
olefile==0.46  
openpyxl==3.0.9  
packaging==21.3  
pandas  
pycparser==2.21  
pyparsing==3.0.7  
python-dateutil==2.8.2  
pytz==2021.3  
six==1.16.0  
xlrd==1.1.0  

You will also need to install my hacked version of bids2nda

```
pip3 install git+https://github.com/evansorenson/bids2nda.git#egg=bids2nda
```

Finally, install nda-tools (needed for EEG uploads via command line) 

```
pip3 install nda-tools
```

### Adding subjects to mapping.txt

This script will no work without a proper mapping.txt file. You will find the mapping.txt in the upload directory. Put all subjects you want to upload in the file. You must be the SubjectID followed by a space, a dash, a space, and finally the GUID.

```
SubjectID - GUID
10001 - NDAR893FAKE
```

### Running the script

```
[esorenson@thalamege scripts]$ python3 nda_upload/nda_upload.py -h
usage: [Upload Dir] [Task] [--copy_dir Copy Dir] [--eeg] [--upload 'Title of NDA Submission' 'Description of NDA Submission'] ... 

NDA Upload subject

positional arguments:
  upload_dir            directory to upload
  task                  task to match in subject info excel column

optional arguments:
  -h, --help            show this help message and exit
  --copy_dir COPY_DIR   directory to copy files from
  --eeg                 for eeg files a different workflow using manifest
                        structure is needed, default is false
  --upload [UPLOAD [UPLOAD ...]]
                        No data preparation. Upload data to NDA. Takes 2
                        arguments: title and description for upload.
```

*If you get a permission denied error for the subject info sheet, run this command and enter your password*  
usr/bin/kinit

Run the script to prepare the data. If you have new subjects to add, use the --copy_dir to copy them from their normal BIDS location into their NDA upload directory  

The script will output a data structure text file (image03 for MRI and eeg_subjects for EEG). This is what is used to upload to NDA. The EEG upload will also generate manifest files, mapping data files to each subject.

**When running upload for EEG, you will need to be on your own computer (Thalamege can't connect to NDA via ssh) and have the rdss and lss drives mounted**

## Example Script calls

### ThalHi fMRI

python3 nda_upload/nda_upload.py /mnt/nfs/lss/lss_kahwang_hpc/data/NDA_uploads/thalHi_mri ThalHi --copy_dir /data/backed_up/shared/ThalHi_MRI_2020/BIDS

__Upload can be done from HTML validation tool__

### ThalHi Resting-state & Diffusion

python3 nda_upload.py /mnt/nfs/lss/lss_kahwang_hpc/data/NDA_uploads/thalHi_diffusion ThalHi --copy_dir /data/backed_up/shared/ThalHi_Diffusion/BIDS/

__Upload can be done from HTML validation tool__


### ThalHi EEG

python nda_upload.py /mnt/nfs/lss/lss_kahwang_hpc/data/NDA_uploads/thalHi_eeg "ThalHi OneTree" --eeg --copy_dir /data/backed_up/shared/ThalHi_EEG/BIDS

__Upload must be done directly from own computer (not connected to Thalamege) and with rdss and lss mounted__

python nda_upload.py /Volumes/lss_kahwang_hpc/data/NDA_uploads/thalHi_eeg "ThalHi OneTree" --eeg --upload "ThalHi EEG Data" "EEG data for hierarchical task control"

### Alpha EEG

python nda_upload.py /mnt/cifs/rdss/rdss_kahwang/Alpha/AlphaStudy_new/Data/BIDS "Alpha Sledgehammer Foster EEG ses 1" --eeg

__Upload must be done directly from own computer (not connected to Thalamege) and with rdss and lss mounted__

python nda_upload.py /Volumes/rdss_kahwang/Alpha/AlphaStudy_new/Data/BIDS "Alpha Sledgehammer Foster EEG ses 1" --eeg --upload "ThalHi EEG Data" "EEG data for hierarchical task control" 
